# Description du Notebook

Ce notebook permet d'entraîner un modèle de NER en français avec spaC en se basant les infos de voyage. 

Les étapes incluent : 
- l'importation, le nettoyage et la préparation des données annotées,
- l'entraînement du modèle. 

Ensuite, on fait un comparaison de performances entre le modèle personnalisé et le modèle spaCy préentraîné, avec des visualisations pour évaluer la diversité et le nombre d'entités détectées par chaque modèle.


In [2]:
import pandas as pd
import spacy
from spacy.tokens import DocBin
from spacy.util import minibatch
from spacy.training.example import Example

# Charger les jeux de données
text_data = pd.read_csv('C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/raw/generated_dataset/text.csv')
token_data = pd.read_csv('C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/raw/generated_dataset/token.csv')
print("finished")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 14, saw 2


In [ ]:
def prepare_ner_data(df):
    nlp_blank = spacy.blank("fr")  # Modèle vierge pour le français
    doc_bin = DocBin()
    
    for _, row in df.iterrows():
        text = row['text']  # Colonne contenant les phrases
        entities = row['entities']  # Colonne avec les entités (ex: [(start, end, label)])
        
        doc = nlp_blank.make_doc(text)
        ents = []
        for start, end, label in entities:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is not None:
                ents.append(span)
        
        doc.ents = ents
        doc_bin.add(doc)
    
    return doc_bin

# Préparer les données et les sauvegarder pour l’entraînement
doc_bin = prepare_ner_data(token_data)
doc_bin.to_disk("ner_training.spacy")

In [ ]:
# Charger un modèle vierge SpaCy
nlp = spacy.blank("fr")

# Configurer le pipeline NER
ner = nlp.add_pipe("ner")

# Ajouter les labels trouvés dans les données
for _, row in token_data.iterrows():
    for _, _, label in row['entities']:
        ner.add_label(label)

# Charger les données d’entraînement
doc_bin = DocBin().from_disk("ner_training.spacy")
train_data = list(doc_bin.get_docs(nlp.vocab))

# Entraînement du modèle
optimizer = nlp.begin_training()
for epoch in range(10):
    losses = {}
    batches = minibatch(train_data, size=8)
    for batch in batches:
        examples = [Example.from_dict(nlp.make_doc(doc.text), {"entities": [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]}) for doc in batch]
        nlp.update(examples, drop=0.3, losses=losses, sgd=optimizer)
    print(f"Epoch {epoch}, Losses: {losses}")

# Sauvegarde du modèle entraîné
nlp.to_disk("nlp/models/spacy/ner_model")


In [ ]:
def prepare_text_classification_data(df):
    texts = df['text'].tolist()
    labels = df['label'].tolist()  # On suppose qu'il y a une colonne "label" avec les intentions
    
    return [(text, label) for text, label in zip(texts, labels)]

# Préparer les données
train_data = prepare_text_classification_data(text_data)


In [ ]:
from spacy.pipeline.textcat import Config, SingleLabelCategorizer

# Charger un modèle vierge
nlp = spacy.blank("fr")

# Configurer le pipeline de Text Classification
textcat = nlp.add_pipe("textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})

# Ajouter les étiquettes de classes
for label in set(text_data['label']):
    textcat.add_label(label)

# Préparation des données d'entraînement
train_data = [(nlp.make_doc(text), {"cats": {label: label == label}}) for text, label in train_data]

# Entraînement du modèle
optimizer = nlp.begin_training()
for epoch in range(10):
    losses = {}
    batches = minibatch(train_data, size=8)
    for batch in batches:
        nlp.update(batch, sgd=optimizer, losses=losses)
    print(f"Epoch {epoch}, Losses: {losses}")

# Sauvegarder le modèle entraîné
nlp.to_disk("nlp/models/spacy/textcat_model")


In [ ]:
from sklearn.metrics import classification_report

def evaluate_ner(nlp, examples):
    true_labels = []
    pred_labels = []

    for example in examples:
        doc = nlp(example.text)
        true_labels.extend([ent.label_ for ent in example.ents])
        pred_labels.extend([ent.label_ for ent in doc.ents])
    
    print(classification_report(true_labels, pred_labels))

# Charger les données de test et le modèle NER
nlp_ner = spacy.load("nlp/models/spacy/ner_model")
evaluate_ner(nlp_ner, test_data)


In [ ]:
# Charger les modèles
nlp_ner = spacy.load("nlp/models/spacy/ner_model")
nlp_textcat = spacy.load("nlp/models/spacy/textcat_model")

def process_request(text):
    # Étape 1 : Détection de l'intention
    doc = nlp_textcat(text)
    intent = max(doc.cats, key=doc.cats.get)
    
    # Étape 2 : Extraction des entités
    doc = nlp_ner(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    return intent, entities

# Exemple
text = "Je veux aller de Nantes à Marseille"
intent, entities = process_request(text)
print(f"Intention : {intent}")
print(f"Entités : {entities}")
